# U-Net Experiment 2: Exploring benefit of tile-overlapping

### 0. Helper Classes

In [1]:
import numpy as np
from tensorflow import keras
import pickle


class EvaluationMetrics:
    """
        This class calculates and summarizes evaluation metrics based on the predicted and true labels.
    """

    def __init__(self, x_train, x_val, x_test, y_train, y_val, y_test, y_pred, training_dates, validation_dates, testing_dates, tile_size, step_size,
                 run_count):
        self.class_statistics = self.get_statistics(x_train, x_val, x_test, y_train, y_val, y_test)

        self.training_dates = training_dates
        self.validation_dates = validation_dates
        self.testing_dates = testing_dates
        self.tile_size = tile_size
        self.step_size = step_size
        self.run_count = run_count

        self.jacard = self.jacard_coef(y_test, y_pred)



        self.conf_matrix_land = self.confusion_matrix(y_test, y_pred, 2)
        self.conf_matrix_valid = self.confusion_matrix(y_test, y_pred, 1)
        self.conf_matrix_invalid = self.confusion_matrix(y_test, y_pred, 0)

        self.precision_land = self.precision(self.conf_matrix_land)
        self.sensitivity_recall_land = self.sensitivity_recall(self.conf_matrix_land)
        self.specificy_land = self.specificy(self.conf_matrix_land)

        self.precision_valid = self.precision(self.conf_matrix_valid)
        self.sensitivity_recall_valid = self.sensitivity_recall(self.conf_matrix_valid)
        self.specificy_valid = self.specificy(self.conf_matrix_valid)

        self.precision_invalid = self.precision(self.conf_matrix_invalid)
        self.sensitivity_recall_invalid = self.sensitivity_recall(self.conf_matrix_invalid)
        self.specificy_invalid = self.specificy(self.conf_matrix_invalid)

        self.f1_land = self.f1_scores(self.conf_matrix_land)
        self.f1_invalid = self.f1_scores(self.conf_matrix_invalid)
        self.f1_valid = self.f1_scores(self.conf_matrix_valid)

    def jacard_coef(self, y_true, y_pred):
        y_true_f = keras.backend.flatten(y_true)
        y_pred_f = keras.backend.flatten(y_pred)

        intersection = keras.backend.sum(y_true_f * y_pred_f)
        return (intersection + 1.0) / (
                keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) - intersection + 1.0
        )  #todo reason for +1?

    def jacard_rounding_issue(self, y_true, y_pred):
        # revert one hot encoding => binary tensor [0, 0, 1] back to label [2] (3D array to 2D array)
        label_map_true = np.argmax(y_true, axis=-1)
        label_map_pred = np.argmax(y_pred, axis=-1)
        # convert 2D array into 1D array
        flatten_true = np.reshape(label_map_true, (-1,))
        flatten_pred = np.reshape(label_map_pred, (-1,))
        # one hot encoding
        one_hot_true = np.eye(3)[flatten_true]
        one_hot_pred = np.eye(3)[flatten_pred]
        # calculate intersection (A geschnitten B)
        intersection = np.sum(one_hot_true * one_hot_pred)
        # calculate union (a u B, A vereint B)
        union = len(one_hot_true) + len(one_hot_pred) - intersection
        # return jacard coefficient
        return (intersection + 1) / (union + 1)

    def confusion_matrix(self, y_true, y_pred, label):
        true_positives = 0
        false_positives = 0
        true_negatives = 0
        false_negatives = 0

        # revert one hot encoding => binary tensor [0, 0, 1] back to label [2] (3D array to 2D array)
        label_map_true = np.argmax(y_true, axis=-1)
        label_map_pred = np.argmax(y_pred, axis=-1)
        # convert 2D array into 1D array
        flatten_true = np.reshape(label_map_true, (-1,))
        flatten_pred = np.reshape(label_map_pred, (-1,))

        tp_mask = (flatten_true == flatten_pred) & (flatten_true == label)
        true_positives = np.count_nonzero(tp_mask)

        fn_mask = (flatten_true == label) & (flatten_pred != label)
        false_negatives = np.count_nonzero(fn_mask)

        fp_mask = (flatten_true != label) & (flatten_pred == label)
        false_positives = np.count_nonzero(fp_mask)

        tn_mask = (flatten_true != label) & (flatten_pred != label)
        true_negatives = np.count_nonzero(tn_mask)

        return {
            'true_positives': true_positives,
            'false_positives': false_positives,
            'true_negatives': true_negatives,
            'false_negatives': false_negatives
        }

    def precision(self, conf_matrix):
        return conf_matrix['true_positives'] / (conf_matrix['true_positives'] + conf_matrix['false_positives'])

    def sensitivity_recall(self, conf_matrix):
        return conf_matrix['true_positives'] / (conf_matrix['true_positives'] + conf_matrix['false_negatives'])

    def negative_predictive(self, conf_matrix):
        return conf_matrix['true_negatives'] / (conf_matrix['true_negatives'] + conf_matrix['false_negatives'])

    def specificy(self, conf_matrix):
        return conf_matrix['true_negatives'] / (conf_matrix['true_negatives'] + conf_matrix['false_positives'])

    def f1_scores(self, conf_matrix):
        prec = self.precision(conf_matrix)
        recall = self.sensitivity_recall(conf_matrix)
        return 2 * prec * recall / (prec + recall)

    def print_metrics(self):
        print(f'jacard index: {self.jacard} \n')

        print(f'precision_land: {self.precision_land}')
        print(f'precision_valid: {self.precision_valid}')
        print(f'precision_invalid: {self.precision_invalid} \n')

        print(f'recall_invalid_land: {self.sensitivity_recall_land}')
        print(f'recall_invalid_land: {self.sensitivity_recall_valid}')
        print(f'recall_invalid_land: {self.sensitivity_recall_invalid} \n')

        print(f'specificy_invalid_land: {self.specificy_land}')
        print(f'specificy_invalid_valid: {self.specificy_valid}')
        print(f'specificy_invalid_invalid: {self.specificy_invalid} \n')

        print(f'f1_land: {self.f1_land}')
        print(f'f1_invalid: {self.f1_invalid}')
        print(f'f1_valid: {self.f1_valid}')

        print(f'Training dates: {self.training_dates}, validation dates: {self.validation_dates}, testing dates: {self.testing_dates}')
        print(f'Number of run: {self.run_count}, tile_size: {self.tile_size}, step_size: {self.step_size}')

    def save_to_file(self):
        file_name = f'../metrics/{self.tile_size}_{self.step_size}_{self.run_count}.pkl'
        with open(file_name, 'wb') as file:
            pickle.dump(self, file)

    def get_label_count(self, array):
        revert_one_hot = np.argmax(array, (-1))
        flatten = np.reshape(revert_one_hot, (-1))
        unique_vals, counts = np.unique(flatten, return_counts=True)
        label_count = {}
        for val, count in zip(unique_vals, counts):
            label_count[f'{val}'] = count
        return label_count

    def get_statistics(self, x_train, x_val, x_test, y_train, y_val, y_test):
       return {'y_train': self.get_label_count(y_train),
                 'y_val': self.get_label_count(y_val), 'y_test': self.get_label_count(y_test)}
    # todo add pixel accuracy


The goal of this notebook is to evaluate whether tile-overlapping has an effect on the resulting model.
We will use the exact same setup as in experiment 1 except for the dataset. Each tile has a size of (256, 256) and the step size is 256 to exclude overlap.

### 1. Loading + Preparing Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! ls
%cd drive/MyDrive/MachineLearning/Geospatial_ML
! ls

drive  sample_data
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
architecture.drawio  evaluation  notebooks     README.md
Copy_of_unet.ipynb   models	 prepare_data  requirements.txt


In [4]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    concatenate,
    Conv2DTranspose,
    Dropout,
    UpSampling2D
)
from keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [5]:
data_directory = "../data_colab/256_256"

y_train  = np.load(os.path.join(data_directory, '2022_06_20.npz'))['y_mask']
x_train  = np.load(os.path.join(data_directory, '2022_06_20.npz'))['x_input']

y_val = np.load(os.path.join(data_directory, '2022_07_10.npz'))['y_mask']
x_val = np.load(os.path.join(data_directory, '2022_07_10.npz'))['x_input']

y_test = np.load(os.path.join(data_directory, '2022_07_25.npz'))['y_mask']
x_test = np.load(os.path.join(data_directory, '2022_07_25.npz'))['x_input']

print(y_train.shape)
print(x_train.shape)

print(y_val.shape)
print(x_val.shape)

print(y_test.shape)
print(x_test.shape)

(761, 256, 256)
(761, 256, 256, 5)
(822, 256, 256)
(822, 256, 256, 5)
(761, 256, 256)
(761, 256, 256, 5)


In [6]:
def normalizing(X, y):

  print(y.shape)
  y_one_hot =  np.array([tf.one_hot(item, depth=3).numpy() for item in y])
  print(y_one_hot.shape)
  X_normal = X/255
  return X_normal, y_one_hot

In [7]:
x_train, y_train = normalizing(x_train, y_train)

X_val, y_val = normalizing(x_val, y_val)

x_test, y_test = normalizing(x_test, y_test)


(761, 256, 256)
(761, 256, 256, 3)
(822, 256, 256)
(822, 256, 256, 3)
(761, 256, 256)
(761, 256, 256, 3)


### 2. Compiling the model

In [8]:
def unet_2d(input_shape, num_classes):

    # Define the input layer
    inputs = Input(input_shape)

    # Downsample layers
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Upsample layers
    up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

    # Output layer
    output = Conv2D(num_classes, (1, 1), activation='softmax')(conv7)

    # Define the model
    model = Model(inputs=[inputs], outputs=[output])

    return model

In [9]:
model = unet_2d(input_shape=(256, 256, 5), num_classes=3)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  2944        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

### 3. Execute trainigs + saving results

In [10]:
tile_size = 256
step_size = 256
saving_path = 'experiment_2'
training_dates = '2022_06_20'
validation_dates = '2022_07_10'
testing_dates = '2022_07_25'

In [11]:
def execute_training(count):
  print(f'Start training number {count}')
  model = unet_2d(input_shape=(256, 256, 5), num_classes=3)
  model.compile(optimizer='adam',
                loss=categorical_crossentropy,
                metrics=['accuracy']) # ??? alternatives

  early_stop = EarlyStopping(monitor='accuracy', patience=5) 

  model_history = model.fit(x=x_train, y=y_train, epochs=100, validation_data=(x_val, y_val), callbacks=[early_stop])

  # saving model
  model_name = f'{tile_size}_{step_size}_run_{count}'
  model.save(f'../models/{saving_path}/model_{model_name}.h5')

  # saving model history
  with open(f'../models/{saving_path}/history_{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(model_history.history, file_pi)

  # making predictions
  pred_test = model.predict(x_test)
  pred_val = model.predict(x_val)
  pred_train = model.predict(x_train)


  # calculating metrics
  metrics_test = EvaluationMetrics(x_train, x_val, x_test, y_train, y_val, y_test, pred_test, training_dates, validation_dates, testing_dates, tile_size, step_size, count)
  metrics_val = EvaluationMetrics(x_train, x_val, x_test, y_train, y_val, y_val, pred_val, training_dates, validation_dates, testing_dates, tile_size, step_size, count)
  metrics_train = EvaluationMetrics(x_train, x_val, x_test, y_train, y_val, y_train, pred_train, training_dates, validation_dates, testing_dates, tile_size, step_size, count)

  # saving metrics
  with open(f'../metrics/{saving_path}/metrics_test{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(metrics_test, file_pi)
  with open(f'../metrics/{saving_path}/metrics_val{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(metrics_val, file_pi)
  with open(f'../metrics/{saving_path}/metrics_train{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(metrics_train, file_pi)

  return metrics_test, metrics_val, metrics_train

In [12]:
all_metrics_test = []
all_metrics_val = []
all_metrics_train = []


for i in range(0,10):
  metrics_test, metrics_val, metrics_train = execute_training(i)
  all_metrics_test.append(metrics_test)
  all_metrics_val.append(metrics_val)
  all_metrics_train.append(metrics_train)

Start training number 0
Epoch 1/100
24/24 [==============================] - 31s 539ms/step - loss: 0.5011 - accuracy: 0.7933 - val_loss: 21.5792 - val_accuracy: 0.9641
Epoch 2/100
24/24 [==============================] - 6s 260ms/step - loss: 0.1755 - accuracy: 0.9516 - val_loss: 17.0958 - val_accuracy: 0.9680
Epoch 3/100
24/24 [==============================] - 6s 259ms/step - loss: 0.1023 - accuracy: 0.9685 - val_loss: 15.3985 - val_accuracy: 0.9733
Epoch 4/100
24/24 [==============================] - 6s 259ms/step - loss: 0.0848 - accuracy: 0.9716 - val_loss: 16.5479 - val_accuracy: 0.9729
Epoch 5/100
24/24 [==============================] - 6s 259ms/step - loss: 0.0785 - accuracy: 0.9723 - val_loss: 13.1442 - val_accuracy: 0.9753
Epoch 6/100
24/24 [==============================] - 6s 259ms/step - loss: 0.0710 - accuracy: 0.9742 - val_loss: 11.7623 - val_accuracy: 0.9734
Epoch 7/100
24/24 [==============================] - 6s 260ms/step - loss: 0.0689 - accuracy: 0.9743 - val_loss

NameError: ignored

### 4. Results

In [ ]:
for idx, metric in enumerate(all_metrics_test):
  print(f'========= RUN {idx + 1} ============')
  print(f'TEST DATA')
  metric.print_metrics()
  print()
  print(f'VALIDATION DATA')
  all_metrics_val[idx].print_metrics()
  print(f'TRAINING DATA')
  all_metrics_train[idx].print_metrics()
  print()
  print()



In [ ]:
def get_mean_jacard(all_metrics):
  jacard_array = []
  for idx, metric in enumerate(all_metrics):
    print(metric.jacard)
    jacard_array.append(metric.jacard)

  print()
  print(f'Mean jacard index: {sum(jacard_array)/10}')
  print()
  print(f'Worst index: {min(jacard_array)}')
  print(f'Best index: {max(jacard_array)}')
  print(f'Variance: {max(jacard_array)-min(jacard_array)}')

print('============ TEST DATA ===================')
get_mean_jacard(all_metrics_test)
print()

print('============ VALIDATION DATA ===================')
get_mean_jacard(all_metrics_val)
print()

print('============ TRAINING DATA ===================')
get_mean_jacard(all_metrics_train)
print()

execution time: ~ 60 min

As the mean jacard index is significatly higher and the variance lower compared to the experiment_1 where we used overlap (step_size: 200). We will continue our next experiments without tile overlapping.



tf.Tensor(0.955727, shape=(), dtype=float32)

tf.Tensor(0.9516739, shape=(), dtype=float32)

tf.Tensor(0.7542591, shape=(), dtype=float32)

tf.Tensor(0.9586384, shape=(), dtype=float32)

tf.Tensor(0.85888374, shape=(), dtype=float32)

tf.Tensor(0.97928536, shape=(), dtype=float32)

tf.Tensor(0.9624236, shape=(), dtype=float32)

tf.Tensor(0.9534589, shape=(), dtype=float32)

tf.Tensor(0.9385002, shape=(), dtype=float32)

tf.Tensor(0.97643083, shape=(), dtype=float32)


Mean jacard index: 0.9289280772209167

Worst index: 0.7542591094970703

Best index: 0.9792853593826294

Variance: 0.22502624988555908


start: 15:25

